In [1]:
def date_generate(stock, N, TIME):    


    import pandas as pd
    import numpy  as np
#    N_close = 'Close12:00:00' #тут выбираем от какого фрейма считаем профиты
#    N = 16 #ЧИСЛО БАРОВ ДО TIME
#    TIME = '12:00' # ОТ КАКОГО ВРЕМЕНИ СЧИТАЕМ N БАРОВ НАЗАД

    N_close = 'Close' + TIME # ЭТО ЧТОБЫ ПРОФИТЫ СЧИТАТЬ

    df = pd.read_excel("E:/Host/Data/Database/from_Wealth/{}.xlsx".format(pp), decimal='.', sep = ',', encoding='latin-1', parse_dates=['Date', 'Time'])

    #СОЗДАЕМ 1 БЛОК (ВЫТАСКИВАЕМ ПРОМЕЖУТКИ ВРЕМЕНИ)

    # создали словарь с df по часам и первому-последнему бару
    history_all = {}
    for tt in ['11:00:00','12:00:00','13:00:00','14:00:00','15:00:00','16:00:00','17:00:00']:
        history_all[f'{tt}'] = df[df['Time'] == tt]
    history_all['last_bar']   = df[df['Date'] != df['Date'].shift(-1)]  
    history_all['first_bar']  = df[df['Date'] != df['Date'].shift( 1)]   

    #смержили все в одну таблицу
    merg_all = pd.DataFrame(df['Date'])
    merg_all.drop_duplicates('Date', inplace = True)
    for i in ['first_bar', '11:00:00','12:00:00','13:00:00','14:00:00','15:00:00','16:00:00','17:00:00', 'last_bar']:
        #display(history_all[i].shape)
        #print('drop')
        history_all[i].drop_duplicates('Date', inplace = True)
        #display(history_all[i].shape)
        #display(history_all[i])
        merg_all = pd.merge(merg_all, history_all[i][['Date','Close']], on='Date', how='right', suffixes=('', i))

    merg_all.dropna(inplace=True)
    merg_all.sort_values(by = ['Date'], inplace = True)

    merg_all['Proft1000'] = (merg_all['Close']        .shift(-1)/merg_all[N_close]-1)*100
    merg_all['Proft1300'] = (merg_all['Close13:00:00'].shift(-1)/merg_all[N_close]-1)*100
    merg_all['Proft1830'] = (merg_all['Closelast_bar'].shift(-1)/merg_all[N_close]-1)*100

    merg_all['Min10']          = (merg_all[N_close]/merg_all['Closelast_bar'].shift(1)-1)*100
    merg_all['Min10_1300']     = (merg_all[N_close]/merg_all['Close13:00:00'].shift(1)-1)*100
    merg_all['Min10_1000']     = (merg_all[N_close]/merg_all['Close']        .shift(1)-1)*100
    merg_all['GEP']            = (merg_all['Close']/merg_all['Closelast_bar'].shift(1)-1)*100

    merg_all.dropna(inplace=True)    #дропаем ибо косяки при мерже с 16 рядами



    # СОЗДАЕМ 2 БЛОК ЭТО ПОСЛЕДНИЕ N БАРОВ ОТ КАКОГО ТО ВРЕМЕНИ

    # Создаем блоки утра namber = 16 
    all_N = pd.DataFrame()

    df[TIME] = np.where(df.Time == TIME, 1, 0) #СОЗДАЛИ СТОЛБЕЦ ГДЕ 1 ЕСЛИ ВРЕМЯ СОВПАЛО С TIME 
    for i in range(N-1, len(df)):
         if df.loc[i][TIME] == 1:
            df_N = df.loc[i-(N-1):i]
            all_N = all_N.append(df_N)
    display(all_N.shape)        
    display(len(all_N)/(N))  

    #create data+16close_morning for DATA
    #cобираем по N reshape Close
    W = "Close" #что берем close high low...можно все но нужно 3 мерный нампай
    close = np.array(all_N[W])
    close = np.reshape(close, (int(len(all_N)/N), N))
    #cобираем по N reshape Date
    data = np.array(all_N['Date'])
    data = np.reshape(data, (int(len(all_N)/N), N))
    #cобираем по N reshape Time
    time = np.array(all_N['Time'])
    time = np.reshape(time, (int(len(all_N)/N), N))


    #дата time последняя в ряду
    data = data[:,-1]
    time = time[:,-1]

    #Обьединяем date and close
    data_close = pd.concat([pd.DataFrame(data, columns=['Date']), pd.DataFrame(time, columns=['Time']), pd.DataFrame(close)], axis=1)
    data_close.shape

    ## оке но как насчет своих индикаторов для ряда в 16 последних?

    #add some things
    #ОТДЕЛЬНЫЙ БЛОК КАК ИЗ РЯДА В 16  ВЫТАЩИТЬ КОЕ ЧТО

    from sklearn.preprocessing import MinMaxScaler

    series = data_close.iloc[:,-N:] #ОБЯЗАТЕЛЬНО ЗАДАТЬ ТОЛЬКО ГОЛЫЕ date without time and date 
    series['mean'] = series.iloc[:,0:N].mean(axis=1)
    series['max']  = series.iloc[:,0:N].max(axis=1)
    series['min']  = series.iloc[:,0:N].min(axis=1)

    scaler = MinMaxScaler(feature_range=(-1, 1))
    a = series.iloc[:, :-3]
    a = scaler.fit_transform(a.T)

    #ДОБАВИЛИ НОВЫЕ ФИЧИ
    series['MinMaxScaler'] = a[-1]
    series['Range'] = (series['max']/series['min']-1)*100
    series['CltoMax'] = (series[15]/series['max']-1)*100
    series['CltoMin'] = (series[15]/series['min']-1)*100

    #ДОБАBИЛИ В DATE
    data_close   = pd.concat([data_close,   series.iloc[:,-4:]], axis=1)

    #4 блок
    df4 = pd.read_excel("E:/Host/Data/Database/from_Wealth/{}.xlsx".format(pp), decimal='.', sep = ',', encoding='latin-1', parse_dates=['Date', 'Time'])

    #может стоит убрать первый бар или брать по клозу, ибо зай и лоу 1 бара слишком волатильны
    #Long

    k1 = []
    for i in range(len(df4)):
        if df4.iloc[i,0].date() != df4.iloc[i-1,0].date():
            k = 1
        else:
            k += 1
        k1.append(k)
    df4['Long'] = k1

    #Min

    min_all = []
    for i in range(len(df4)):    

        f = df4['Long'][i]
        min = (df4['Low'][i+1-f:i+1]).min()
        min_all.append(min)
    df4['min_day'] = min_all


    #Max
    max_all = []
    for i in range(len(df4)):    

        f = df4['Long'][i]
        max = (df4['High'][i+1-f:i+1]).max()
        max_all.append(max)
    df4['max_day'] = max_all

    df4['Cl/min_day']      = (df4['Close']/df4['min_day']-1)*100
    df4['Cl/max_day']      = (df4['Close']/df4['max_day']-1)*100
    df4['max_day-min_day'] = ((df4['max_day']-df4['min_day'])/df4['Close'])*100


    #####мердж пока без TA

    df_merg_1 = pd.merge(data_close, merg_all, how='inner', on=['Date'])
    df4_for_merg = df4[df4['Time'] == TIME]
    df_merg_2 = pd.merge(df_merg_1, df4_for_merg, how='inner', on=['Date'])

    display(merg_all.shape)
    display(data_close.shape)
    #display(TA.shape)
    display(df4.shape)
    print(100*'**')
    display(df_merg_1.shape)
    df_merg_2.drop_duplicates('Date')
    display(df_merg_1.shape)
    display(df_merg_2.shape)
    df_merg_2.drop_duplicates('Date', inplace=True)
    display(df_merg_2.shape)
    df_merg_2.dropna(inplace=True)
    display(df_merg_2.shape)
    
    print('df_merg_2, data_close, merg_all')
    return(df_merg_2, data_close, merg_all)

In [33]:
%%time

df_all = pd.DataFrame()
for pp in ['GAZP', 'GMKN', 'LKOH', 'ROSN', 'SBER', 'VTBR']:
    date_generate(pp, 16, '12:30') # date_generate(stock, N, TIME)
    df_merg_2['Symbol'] = pp
    df_all = df_all.append(df_merg_2)

C:\Users\afecn\anaconda3.1\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(48752, 8)

3047.0

(3015, 10)

(3047, 22)

...

(2686, 31)

(2690, 43)

(2684, 43)

(2684, 43)

df_merg_2, data_close, merg_all
Wall time: 7min 20s


In [39]:
df_all

,Date,Time_x,0,1,2,3,4,5,6,7,...,Close_y,Volume,Long,min_day,max_day,Cl/min_day,Cl/max_day,max_day-min_day,symbol,Symbol
0,2006-01-24,2020-09-11 12:00:00,220.250000,220.440002,219.449997,219.399994,219.610001,219.490005,220.000000,218.960007,...,220.250000,149187,7,219.660004,220.869995,0.268595,-0.280706,0.549372,VTBR,GAZP
1,2006-01-25,2020-09-11 12:00:00,222.199997,222.199997,222.600006,222.449997,222.869995,222.899994,223.000000,223.889999,...,226.110001,796537,7,225.000000,231.000000,0.493334,-2.116883,2.653576,VTBR,GAZP
2,2006-01-26,2020-09-11 12:00:00,228.080002,228.789993,228.729996,228.500000,228.600006,228.449997,228.770004,228.699997,...,227.880005,171180,7,226.449997,229.410004,0.631489,-0.666928,1.298932,VTBR,GAZP
3,2006-01-27,2020-09-11 12:00:00,227.250000,226.699997,225.710007,224.500000,225.500000,225.160004,225.389999,225.160004,...,225.800003,116330,7,225.000000,227.199997,0.355557,-0.616195,0.974312,VTBR,GAZP
4,2006-01-30,2020-09-11 12:00:00,227.550003,227.000000,226.759995,227.899994,227.399994,227.449997,227.500000,227.729996,...,228.000000,333261,7,227.000000,231.070007,0.440529,-1.328605,1.785091,VTBR,GAZP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3033,2018-04-03,2020-09-11 12:00:00,141.210007,141.199997,140.899994,140.600006,141.360001,140.910004,140.320007,140.860001,...,140.490005,255620,9,139.809998,140.990005,0.486380,-0.354635,0.839923,VTBR,VTBR
3034,2018-04-04,2020-09-11 12:00:00,140.460007,140.539993,140.699997,140.699997,140.610001,140.660004,141.130005,140.669998,...,139.520004,1352580,9,139.500000,141.520004,0.014340,-1.413228,1.447824,VTBR,VTBR
3035,2018-04-05,2020-09-11 12:00:00,139.820007,139.570007,139.710007,139.740005,140.070007,140.190002,140.179993,141.199997,...,140.720001,175790,9,140.149994,141.350006,0.406712,-0.445706,0.852766,VTBR,VTBR
3036,2018-04-06,2020-09-11 12:00:00,140.600006,140.190002,140.149994,140.610001,140.309998,140.250000,140.500000,140.139999,...,141.330002,192740,9,139.449997,141.399994,1.348157,-0.049499,1.379747,VTBR,VTBR


In [25]:
TA = pd.read_csv('df3.csv', parse_dates=['Date', 'Time'])

In [29]:
#corection with TA

TA[TIME] = np.where(TA.Time == '2020-09-06 12:30:00', 1, 0) #СОЗДАЛИ СТОЛБЕЦ ГДЕ 1 ЕСЛИ ВРЕМЯ СОВПАЛО С TIME
TA_for_merg = TA[TA[TIME] == 1]
TA_for_merg

,Unnamed: 0,Date,Time,Open,High,Low,Close,Volume,Symbol,ADX_16,...,HT_PHASOR_inphase,HT_PHASOR_quadrature,HT_SINE_sine,HT_SINE_leadsine,BBANDS_upperband,BBANDS_middleband,BBANDS_lowerband,MAMA_mama,MAMA_fama,12:00
17,107,2006-01-26,2020-09-06 12:30:00,227.789993,228.259994,227.539993,228.000000,150271,GAZP,-0.485807,...,-0.221780,0.001705,-0.127640,0.611068,-0.998263,-0.105057,0.990218,-0.339370,-0.341736,1
50,140,2006-01-27,2020-09-06 12:30:00,225.350006,225.690002,225.330002,225.550003,65461,GAZP,-0.187867,...,-0.220212,0.001557,0.949671,0.893019,-0.993943,-0.099638,0.995750,-0.336626,-0.338516,1
83,173,2006-01-30,2020-09-06 12:30:00,227.500000,227.500000,225.110001,225.350006,864927,GAZP,-0.736092,...,-0.221298,0.000484,-0.445184,-0.947964,-0.928022,-0.060391,0.997307,-0.313257,-0.314104,1
116,206,2006-01-31,2020-09-06 12:30:00,222.369995,222.699997,221.690002,222.399994,221180,GAZP,-0.328318,...,-0.218428,0.004453,0.821992,0.983933,-0.952900,-0.124511,0.902667,-0.352053,-0.358310,1
149,239,2006-02-01,2020-09-06 12:30:00,220.729996,222.199997,220.729996,222.000000,282701,GAZP,-0.702989,...,-0.219458,0.004226,0.227375,-0.527807,-0.993386,-0.124212,0.948258,-0.362655,-0.369229,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603728,92111,2018-04-03,2020-09-06 12:30:00,0.052410,0.053100,0.052240,0.052640,6506640,VTBR,-0.486078,...,-0.220899,0.001737,0.587424,0.987618,-0.966202,-0.189899,0.792737,-0.432547,-0.410370,1
603764,92147,2018-04-04,2020-09-06 12:30:00,0.052260,0.052380,0.052250,0.052270,282960,VTBR,-0.300991,...,-0.220899,0.001737,-0.505436,-0.967534,-0.999066,-0.101304,0.998270,-0.333469,-0.340800,1
603800,92183,2018-04-05,2020-09-06 12:30:00,0.053160,0.053160,0.052970,0.053050,1688240,VTBR,-0.364812,...,-0.220899,0.001736,-0.001487,-0.708157,-0.982879,-0.099869,0.983007,-0.337527,-0.355005,1
603836,92219,2018-04-06,2020-09-06 12:30:00,0.054840,0.054860,0.054800,0.054840,624090,VTBR,0.169943,...,-0.220899,0.001737,0.195733,-0.555025,-0.987067,-0.097657,0.991884,-0.344180,-0.372558,1


In [42]:
merg_with_TA = pd.merge(TA_for_merg, df_all, on = ['Date', 'Symbol'], how='inner')

In [43]:
display(df_all.shape)
display(merg_with_TA.shape)
merg_with_TA.drop_duplicates(['Date', 'Symbol'], inplace=True)
display(merg_with_TA.shape)

(18084, 45)

(17621, 124)

(17590, 124)

In [45]:
list(merg_with_TA)

['Unnamed: 0',
 'Date',
 'Time',
 'Open_x',
 'High_x',
 'Low_x',
 'Close',
 'Volume_x',
 'Symbol',
 'ADX_16',
 'ADXR_16',
 'APO_16',
 'AROONOSC_16',
 'BOP_16',
 'CCI_16',
 'CMO_16',
 'DX_16',
 'MFI_16',
 'MINUS_DI_16',
 'MINUS_DM_16',
 'MOM_16',
 'PLUS_DI_16',
 'PLUS_DM_16',
 'PPO_16',
 'ROC_16',
 'ROCP_16',
 'ROCR_16',
 'ROCR100_16',
 'RSI_16',
 'TRIX_16',
 'ULTOSC_16',
 'WILLR_16',
 'HT_DCPERIOD_16',
 'HT_DCPHASE_16',
 'HT_TRENDMODE_16',
 'DEMA_16',
 'EMA_16',
 'HT_TRENDLINE_16',
 'KAMA_16',
 'MA_16',
 'MIDPOINT_16',
 'MIDPRICE_16',
 'SMA_16',
 'T3_16',
 'TEMA_16',
 'TRIMA_16',
 'WMA_16',
 'ATR_16',
 'NATR_16',
 'TRANGE_16',
 'AD_16',
 'ADOSC_16',
 'OBV_16',
 'Symbol.1',
 'AROON_aroondown',
 'AROON_aroonup',
 'MACD_macd',
 'MACD_macdsignal',
 'MACD_macdhist',
 'MACDEXT_macd',
 'MACDEXT_macdsignal',
 'MACDEXT_macdhist',
 'MACDFIX_macd',
 'MACDFIX_macdsignal',
 'MACDFIX_macdhist',
 'STOCH_slowk',
 'STOCH_slowd',
 'STOCHF_fastk',
 'STOCHF_fastd',
 'STOCHRSI_fastk',
 'STOCHRSI_fastd',
 '